In [107]:
#import library to connect to mysql database
import mysql.connector

### Connects to UMLS DB 2020 and fetches the synonyms from CHV table (note: Connect to Umass lowell Global Protect VPN)

In [108]:
import requests
import argparse

#Enter the databse details below
umls_db = mysql.connector.connect(host='', port='',
                                user='', password='', database='')
print(umls_db)
def GetSynonms(search_text):
    mycursor = umls_db.cursor(buffered=True)

    sql = (f"SELECT CUI FROM MRCONSO Where Sab='CHV' and STR='{search_text}'")
    
    mycursor.execute(sql)
    results = mycursor.fetchall()
    field_names = [ col[0] for col in mycursor.description]
    
    a=(results[0][0])
    sql = (f"SELECT STR FROM MRCONSO Where CUI='{a}' and Sab='CHV' LIMIT 30")

    mycursor.execute(sql)
    results = mycursor.fetchall()
    
    return results

### Create Flask app for Web search

In [ ]:
import pysolr
from flask import Flask, jsonify, request
from datetime import datetime

# Connect to Solr
solr = pysolr.Solr('http://localhost:8983/solr/mimic3/', timeout=100)

# Create Flask app
app = Flask(__name__)

# Route to load web page
@app.route('/')
def index():
    print('index function called')
    return app.send_static_file('index.html')

# Route to Search and display results
@app.route('/Search', methods=['POST'])
def Search():
    query=''
    synonms=''
    # Get all inputs
    search_text = request.form.get('query', '')
    expiry_flag_input = request.form.get('expiry_flag')
    start_date = request.form.get('start_date','All')
    end_date = request.form.get('end_date','All')
    icd_codes = request.form.get('icd_codes','All') 
    
    # Get synonyms for text enterd by user
    if(search_text !=''):
        synonms= GetSynonms(search_text)  
        print(type(synonms))
        synonms_list=[]
        for s in synonms:
          synonms_list.append((s)[0].replace(' ', '\\ '))
        query = " text:".join([r for r in synonms_list]) 
        query = " text:"+query

    # Update query based on Hospital Expiry Flag entered by user
    if expiry_flag_input=='Died':
        query= query +" AND expiry_flag: "+ str(1)
    elif expiry_flag_input=='Survived':
        query= query+" AND expiry_flag: "+ str(0)
    else:
        print('expiry_flag_input not set'+ expiry_flag_input)
    

    # Update query based on icd_codes entered by user 
    if(icd_codes!=''):
        icd_list=""
        icd_list=((icd_codes).replace(',', ' AND icd_codes:'))
        if (query!=""):
            query= query+" AND icd_codes:"+ str(icd_list)
        else:
            query= query+" icd_codes:"+ str(icd_list)
        print(icd_codes)
        
     # Update query based on start and end date entered by user   

    if(start_date!='' and end_date!=''):
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        if (query!=""):
            date_str= f' AND chartdate:[{start_date.isoformat()}Z TO {end_date.isoformat()}Z]'
        else:
            date_str= f' chartdate:[{start_date.isoformat()}Z TO {end_date.isoformat()}Z]'
        query=query+date_str
    elif(start_date!=''):
            start_date = datetime.strptime(start_date, '%Y-%m-%d')
            if (query!=""):
                    date_str= f' AND chartdate:[{start_date.isoformat()}Z TO *]'
            else:
                date_str= f' chartdate:[{start_date.isoformat()}Z TO *]'
            query=query+date_str
    elif(end_date!=''):      
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        if (query!=""):
            date_str= f' AND chartdate:[* TO {end_date.isoformat()}Z]'
        else:
            date_str= f' chartdate:[* TO {end_date.isoformat()}Z]'
        query=query+date_str

    # Add params for Solr search
    params = {
    'hl': 'true',
    'hl.simple.pre': '<strong>',
    'hl.simple.post': '</strong>',
    'hl.fl': 'text',
    'hl.snippets': 3,
    'rows': 20
    }
    
    print(query)
    # Perform Solr search
    results = solr.search(query, **params)
    id_list=[]
    
     # store results from solr search into needed format and send it to be displayed
    if len(results)==0:
        results= range(0,1)
        data = [{'id': '',
             'text':'',
             'category': '',
             'hadm_id': '',
             'expiry_flag': '',
             'icd_codes': '',
             'chartdate':'',
             'synonyms':  '',
             'Num_results':"No Notes found, enter different query",
             'solr_query': query,
              'id_list': 'NA'
            
            } for result in results]
    else:
        for result in results:
            id_list.append(result['id'])
            if len(id_list)==10:
                break
        data = [{'id': result['id'],
         'text': result['text'],
         'category': result['category'],
         'hadm_id': result['hadm_id'],
         'expiry_flag': result['expiry_flag'],
         'icd_codes': result['icd_codes'],
         'chartdate':result['chartdate'],
         'synonyms':  synonms,
         'Num_results':len(results),
         'solr_query': query,
         'id_list': id_list
        } for result in results]
    return  jsonify(data)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Mar/2023 03:39:03] "GET /%7B%7B%20url_for('static',%20filename='styles.css')%20%7D%7D HTTP/1.1" 404 -
127.0.0.1 - - [15/Mar/2023 03:39:15] "POST /Search HTTP/1.1" 200 -


expiry_flag_input not setBoth
 chartdate:[2110-01-01T00:00:00Z TO 2120-01-01T00:00:00Z]


127.0.0.1 - - [15/Mar/2023 03:40:23] "POST /Search HTTP/1.1" 200 -


<class 'list'>
expiry_flag_input not setBoth
 text:brain\ malignant\ tumors text:malignant\ brain\ tumor text:brain\ cancer text:brain\ ca text:ca\ brain text:cancer\ of\ the\ brain text:brain\ cancers text:brain\ malignant\ tumors text:malignant\ neoplasm\ of\ brain text:cancer\ of\ brain text:malignant\ neoplasm\ of\ brain
